## Adding the libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math


## Data Cleaning

In [ ]:
# Load the data
file_path = "output.csv"
df = pd.read_csv(file_path)

## Date cleaning

### Dropping rows with null values

In [ ]:
# Drop rows with NaN values
df = df.dropna()

### Changing Dates

In [ ]:
# # Convert 'Date' column to datetime type
# df['Date'] = pd.to_datetime(df['Date'])

# # Subtract 30 days from each date
# df['Date'] = df['Date'] - pd.Timedelta(days=30)

# # Output the updated DataFrame
# print(df['Date'])

### Taxi and average delay Takeoff Data cleaning 

In [ ]:
import re
# Define a function to extract the numeric part from the string
def extract_numeric(text):
    match = re.search(r'\d+', text)
    if match:
        return int(match.group())
    else:
        return None

# Apply the function to clean the "Taxi Landing" column
df['Taxi Landing'] = df['Taxi Landing'].apply(extract_numeric)
df['Taxi Takeoff'] = df['Taxi Takeoff'].apply(extract_numeric)

### Takeoff Conversion from 12 hours format to 24 hours format

In [ ]:
# Define the time-related columns
time_columns = ['Gate Departure Actual', 'Gate Departure Estimated', 'Takeoff Actual', 'Takeoff Estimated', 'Gate Arrival Actual', 'Gate Arrival Estimated', 'Landing Actual', 'Landing Estimated']

# Replace non-time values ('--') with NaN
df[time_columns] = df[time_columns].replace('--', pd.NA)

# Convert time columns to 24-hour format
for column in time_columns:
    # Convert to datetime objects
    df[column] = pd.to_datetime(df[column], errors='coerce')
    # Convert to 24-hour format
    df[column] = df[column].dt.strftime('%H:%M')

# Display the DataFrame after conversion
# print(df[time_columns])
print(df.head())

## Assigning feature values

In [ ]:
# Define features and target
features = ['Takeoff Actual', 'Takeoff Estimated', 'Landing Actual','Landing Estimated']
target = 'Landing Actual'

X = df[features]
y = df[target]
X

## Converting into tensors

In [ ]:

# Convert time columns to numerical format (hours and minutes)
for column in ['Takeoff Actual', 'Takeoff Estimated', 'Landing Actual', 'Landing Estimated']:
    df[column] = pd.to_datetime(df[column]).dt.hour * 60 + pd.to_datetime(df[column]).dt.minute

# Define a function to map categories to numerical values
def map_delay_to_numeric(text):
    if 'Less than 10 minutes' in text:
        return 0
    elif '10-20 minutes' in text:
        return 1
    elif '20-40 minutes' in text:
        return 2
    elif '40 minutes - 1 hour' in text:
        return 3
    elif 'More than 1 hour' in text:
        return 4
    else:
        return -1  # Handle unknown cases if any

# Convert all values in 'Average Delay Takeoff' column to strings
df['Average Delay Takeoff'] = df['Average Delay Takeoff'].astype(str)

# Apply the mapping function to convert categories to numerical values
df['Average Delay Takeoff Numeric'] = df['Average Delay Takeoff'].apply(map_delay_to_numeric)

# Define features and target
features = ['Takeoff Actual', 'Takeoff Estimated', 'Landing Estimated', 'Average Delay Takeoff Numeric']
target = 'Landing Actual'

# Extract features and target
X = df[features]
y = df[target]

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values.reshape(-1, 1), dtype=torch.float32)

# Split the data into training and testing sets
X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Check the first few rows of the modified DataFrame
print(df[['Average Delay Takeoff', 'Average Delay Takeoff Numeric']].head())

## Neural Network Architecture

In [124]:
# Define the correct number of input features
num_input_features = X_train_tensor.shape[1]

# Define neural network architecture
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(num_input_features, 64)  # Input layer
        self.fc2 = nn.Linear(64, 32)                  # Hidden layer
        self.fc3 = nn.Linear(32, 1)                   # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Apply ReLU activation function to first layer
        x = torch.relu(self.fc2(x))  # Apply ReLU activation function to second layer
        x = self.fc3(x)              # Output layer (no activation function)
        return x
    
    @torch.jit.export
    def __str__(self):      # The toString method for the class
        return "<CNN Initialized>"
    
    @torch.jit.export
    def predict(self, take_off_actual: int, take_off_estimated: int, landing_estimated: int, average_delay: int):
        data = [x if x is not None else 0 for x in [take_off_actual, take_off_estimated, landing_estimated, average_delay]]
        new_data_tensor = torch.tensor(data, dtype=torch.float32)
        predicted_arrival_time_tensor = self.forward(new_data_tensor)
        predicted_arrival_time = predicted_arrival_time_tensor.item()

        return predicted_arrival_time


# Instantiate the model
model = NeuralNetwork()
# Make predictions on the training data
model.eval()
with torch.no_grad():
    y_pred_train_tensor = model(X_train_tensor)



## Defining loss function and training the model

In [125]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10000
train_losses = []
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    train_losses.append(loss.item())
    
    # Print loss every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/10000], Loss: 632.3597
Epoch [200/10000], Loss: 607.0744
Epoch [300/10000], Loss: 555.1799
Epoch [400/10000], Loss: 450.5460
Epoch [500/10000], Loss: 367.2429
Epoch [600/10000], Loss: 295.4038
Epoch [700/10000], Loss: 235.6889
Epoch [800/10000], Loss: 189.3632
Epoch [900/10000], Loss: 156.1434
Epoch [1000/10000], Loss: 134.2830
Epoch [1100/10000], Loss: 120.8329
Epoch [1200/10000], Loss: 112.5780
Epoch [1300/10000], Loss: 107.3254
Epoch [1400/10000], Loss: 103.2598
Epoch [1500/10000], Loss: 99.6011
Epoch [1600/10000], Loss: 96.1180
Epoch [1700/10000], Loss: 92.5580
Epoch [1800/10000], Loss: 88.1640
Epoch [1900/10000], Loss: 83.3972
Epoch [2000/10000], Loss: 78.2502
Epoch [2100/10000], Loss: 72.7750
Epoch [2200/10000], Loss: 66.2138
Epoch [2300/10000], Loss: 59.5208
Epoch [2400/10000], Loss: 52.1535
Epoch [2500/10000], Loss: 45.3245
Epoch [2600/10000], Loss: 39.2853
Epoch [2700/10000], Loss: 34.1128
Epoch [2800/10000], Loss: 29.7871
Epoch [2900/10000], Loss: 26.2867
Epoch [30

## Model Evaluation

In [126]:
# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    mse = criterion(y_pred_tensor, y_test_tensor)
    print(f'Mean Squared Error on Test Data: {mse.item():.4f}')

Mean Squared Error on Test Data: 16.2938


## Data Visualisation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(df[features].corr(), annot=True, cmap='Blues')
plt.title("Correlation of Features")
plt.show()

# Plot actual vs. predicted values for training data
plt.figure(figsize=(10, 8))
plt.scatter(y_train_tensor, model(X_train_tensor).detach().numpy(), color='blue', alpha=0.5, label='Training Values')
plt.plot([min(y_train_tensor), max(y_train_tensor)], [min(y_train_tensor), max(y_train_tensor)], linestyle='--', color='blue')  # Diagonal line
plt.xlabel("Actual Time of Arrival (Training)")
plt.ylabel("Predicted Time of Arrival (Training)")
plt.title("Actual vs. Predicted Time of Arrival (Neural Network - Training)")
plt.legend()
plt.grid(True)
plt.show()

# Plot actual vs. predicted values for test data
plt.figure(figsize=(10, 8))
plt.scatter(y_test_tensor, y_pred_tensor.numpy(), color='green', alpha=0.5)
plt.plot([min(y_test_tensor), max(y_test_tensor)], [min(y_test_tensor), max(y_test_tensor)], linestyle='--', color='red')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values (Neural Network - Test)')
plt.grid(True)
plt.show()

# Calculate the absolute difference between actual and estimated time of arrival
arrival_time_difference = torch.abs(y_test_tensor - y_pred_tensor).numpy()

# Visualize the difference
plt.figure(figsize=(10, 6))
plt.hist(arrival_time_difference, bins=10, color='skyblue', edgecolor='black')
plt.xlabel('Absolute Difference between Actual and Estimated Time of Arrival')
plt.ylabel('Frequency')
plt.title('Distribution of Arrival Time Differences')
plt.show()

# Analyze the difference
mean_difference = arrival_time_difference.mean()
max_difference = arrival_time_difference.max()
min_difference = arrival_time_difference.min()

print("Mean difference:", mean_difference)
print("Maximum difference:", max_difference)
print("Minimum difference:", min_difference)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Convert predicted values to numpy arrays
y_pred_train = model(X_train_tensor).detach().numpy().reshape(-1)
y_pred_test = model(X_test_tensor).detach().numpy().reshape(-1)

# Calculate Mean Squared Error (MSE) and Mean Absolute Error (MAE) for training and test data
mse_train = mean_squared_error(y_train_tensor, torch.tensor(y_pred_train, dtype=torch.float32))
mae_train = mean_absolute_error(y_train_tensor, torch.tensor(y_pred_train, dtype=torch.float32))
mse_test = mean_squared_error(y_test_tensor, torch.tensor(y_pred_test, dtype=torch.float32))
mae_test = mean_absolute_error(y_test_tensor, torch.tensor(y_pred_test, dtype=torch.float32))

print("Mean Squared Error (MSE) - Train:", mse_train)
print("Mean Absolute Error (MAE) - Train:", mae_train)
print("Mean Squared Error (MSE) - Test:", mse_test)
print("Mean Absolute Error (MAE) - Test:", mae_test)


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_log_error, explained_variance_score

# R² Score (coefficient of determination) = 1 - (SS_res / SS_tot)
# Measures the proportion of the variance in the dependent variable that is predictable from the independent variables.
r2_train = r2_score(y_train_tensor, y_pred_train_tensor)
r2_test = r2_score(y_test_tensor, y_pred_tensor)

# MAPE = (1/n) * Σ(|actual - predicted| / |actual|) * 100%
# Calculates the mean of the absolute percentage errors between the predicted and actual values 
mape_train = mean_absolute_percentage_error(y_train_tensor, y_pred_train_tensor)
mape_test = mean_absolute_percentage_error(y_test_tensor, y_pred_tensor)

# Clip negative values to zero
y_train_tensor_clipped = np.clip(y_train_tensor, 0, None)
y_pred_train_tensor_clipped = np.clip(y_pred_train_tensor, 0, None)
y_test_tensor_clipped = np.clip(y_test_tensor, 0, None)
y_pred_tensor_clipped = np.clip(y_pred_tensor, 0, None)

# Calculate RMSLE with clipped targets
rmsle_train = np.sqrt(mean_squared_log_error(y_train_tensor_clipped, y_pred_train_tensor_clipped))
rmsle_test = np.sqrt(mean_squared_log_error(y_test_tensor_clipped, y_pred_tensor_clipped))

# Calculate Explained Variance Score = 1 - (Var(y_true - y_pred) / Var(y_true))
# Explained Variance Score measures the proportion to which a mathematical model accounts for the variation (dispersion) of a given data set.
explained_variance_train = explained_variance_score(y_train_tensor, y_pred_train_tensor)
explained_variance_test = explained_variance_score(y_test_tensor, y_pred_tensor)

print("R² Score (Train):", r2_train)
print("R² Score (Test):", r2_test)
print("MAPE (Train):", mape_train)
print("MAPE (Test):", mape_test)
print("RMSLE (Train):", rmsle_train)
print("RMSLE (Test):", rmsle_test)
print("Explained Variance Score (Train):", explained_variance_train)
print("Explained Variance Score (Test):", explained_variance_test)


### Ridge Regression to avoid Overfitting

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Initialize Ridge and Lasso regression models
ridge_model = Ridge(alpha=1.0)  # You can adjust the alpha parameter for regularization strength

# Train Ridge and Lasso models
ridge_model.fit(X_train_tensor, y_train_tensor)

# Predictions
y_pred_ridge_train = ridge_model.predict(X_train_tensor)
y_pred_ridge_test = ridge_model.predict(X_test_tensor)


# Evaluate the models
mse_ridge_train = mean_squared_error(y_train_tensor, y_pred_ridge_train)
mse_ridge_test = mean_squared_error(y_test_tensor, y_pred_ridge_test)


print("Ridge Regression:")
print("Train MSE:", mse_ridge_train)
print("Test MSE:", mse_ridge_test)



In [ ]:

# Histogram: Distribution of taxi landing time with bigger bins
plt.figure(figsize=(8, 6))
sns.histplot(df['Taxi Landing'], bins=50, color='skyblue', edgecolor='black', kde=True)
plt.title('Distribution of Taxi Landing Time')
plt.xlabel('Taxi Landing Time (minutes)')
plt.ylabel('Frequency')

# Histogram: Distribution of taxi takeoff time with bigger bins
plt.figure(figsize=(8, 6))
sns.histplot(df['Taxi Takeoff'], bins=50, color='skyblue', edgecolor='black', kde=True)
plt.title('Distribution of Taxi Takeoff Time')
plt.xlabel('Taxi Takeoff Time (minutes)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Bar chart: Average delay landing by flight number
plt.figure(figsize=(12, 6))
sns.barplot(x='Flight Number', y='Average Delay Landing', data=df, palette='viridis')
plt.title('Average Delay Landing by Flight Number')
plt.xlabel('Flight Number')
plt.ylabel('Average Delay Landing (minutes)')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Assuming y_test_tensor and y_pred_test_tensor are available
# Convert tensor values to numpy arrays
y_test = y_test_tensor.numpy()
y_pred_test = y_pred_tensor.numpy()

# Histogram: Distribution of prediction errors
plt.figure(figsize=(8, 6))
sns.histplot(y_test - y_pred_test, bins=20, color='skyblue', edgecolor='black', kde=True)
plt.title('Distribution of Prediction Errors')
plt.xlabel('Prediction Error (Actual - Predicted)')
plt.ylabel('Frequency')
plt.show()


### Importance of features

In [ ]:

# Get the weights of the input layer
input_layer_weights = model.fc1.weight.detach().numpy()

# Calculate the absolute sum of weights for each feature
feature_importance = np.abs(input_layer_weights).sum(axis=0)  # sum along the columns

# Get the feature names
feature_names = ['Takeoff Actual', 'Takeoff Estimated', 'Landing Estimated', 'Average Delay Takeoff Numeric']

# Plot the feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_names, feature_importance)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
plt.show()

### Individual Prediction

In [127]:

# Sample input data for testing
sample_input_data = {
    'Takeoff Actual': 8 * 60 + 30,  # Takeoff time  (8:30 AM)
    'Takeoff Estimated': 8 * 60 + 40,  # Estimated takeoff time (8:40 AM)
    'Landing Estimated': 9 * 60 + 50,  # Estimated landing time (9:50 AM)
    'Average Delay Takeoff Numeric': 1  # Numeric representation of average delay
}
# features = ['Takeoff Actual', 'Takeoff Estimated', 'Landing Estimated', 'Average Delay Takeoff Numeric']

res = model.predict(8 * 60 + 30, 8 * 60 + 40, 9 * 60 + 50, 1)
print(f"Predicted Actual Time of Arrival using object method: {res}")

Predicted Actual Time of Arrival using object method: 569.512939453125


In [128]:
scripted_model = torch.jit.script(model)
print(model)
scripted_model.save("../trained-model.pt")

<CNN Initialized>
